In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob

folder_path = '/content/drive/MyDrive/data/'

df = pd.DataFrame(columns=['title','url','text'])
file_list = glob.glob(folder_path + "/*.txt")

import re
title_pattern = re.compile(r'TITLE:\s*(.*)LINK', re.IGNORECASE)
link_pattern = re.compile(r'LINK:\s*(\S*)', re.IGNORECASE)


for i, txt_file in enumerate(file_list):
    with open(txt_file, 'r') as file:
        text = file.read()
        title_match = title_pattern.search(text)
        if title_match:
            title = title_match.group(1)

        link_match = link_pattern.search(text)
        if link_match:
            link = link_match.group(1)
        remaining_text = text.split('\n', 1)[1]
        df.loc[i] = [title, link, remaining_text]

In [ ]:
!pip install locationtagger -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 33.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
from brat_parser import get_entities_relations_attributes_groups
entities, relations, attributes, groups = get_entities_relations_attributes_groups("/content/drive/MyDrive/data/0.ann")

In [ ]:
import nltk
import spacy

nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import locationtagger

def location_in_text(sample_text):
  place_entity = locationtagger.find_locations(text = sample_text)
  return [place_entity.countries, place_entity.regions, place_entity.cities]

### Baseline Locations

In [ ]:
places_df = pd.DataFrame(columns=['countries','states','cities'])
for i in range(len(df)):
  places_df.loc[i] = location_in_text(df.text[i]+df.title[i])

In [ ]:
places_df.head()

,countries,states,cities
0,[],"[Washington, New Jersey]","[Washington, Paterson, Hope, Allendale, Washin..."
1,[],"[Illinois, New York]","[New York, White House, Barr]"
2,[],[],[]
3,[],[Kuala Lumpur],"[Kuala Lumpur, Us, Four]"
4,[Slovakia],[],"[Bratislava, Nitra, English]"


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import string

stop_words = stopwords.words('english')
def preprocess(text):
    if pd.isna(text):
      return ''
    text = text.strip()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if not word in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    text = ' '.join(tokens)
    return text

### BERT NER Locations


In [ ]:
!pip install transformers[sentencepiece]
!apt install git-lfs

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

In [ ]:
print(df.url[2])

https://www.engadget.com/2018/04/05/dropbox-smart-sync-business-launch/?utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%3A+weblogsinc%2Fengadget+%28Engadget%29


In [ ]:
nlp(df.title[0]+df.text[0])

[{'entity_group': 'MISC',
  'score': 0.93269277,
  'word': "Allendale Woman ' s Club Fashion Show To Help Struggling Moms, Children Mother",
  'start': 0,
  'end': 76},
 {'entity_group': 'ORG',
  'score': 0.5833455,
  'word': '##s',
  'start': 76,
  'end': 77},
 {'entity_group': 'ORG',
  'score': 0.8627121,
  'word': 'The Allendale Woman ’ s Club',
  'start': 212,
  'end': 238},
 {'entity_group': 'MISC',
  'score': 0.5210386,
  'word': 'Meredith',
  'start': 254,
  'end': 262},
 {'entity_group': 'PER',
  'score': 0.48878282,
  'word': 'Desmond',
  'start': 263,
  'end': 270},
 {'entity_group': 'ORG',
  'score': 0.573388,
  'word': 'Fig Leaf Boutique',
  'start': 371,
  'end': 388},
 {'entity_group': 'MISC',
  'score': 0.5239181,
  'word': 'May',
  'start': 390,
  'end': 393},
 {'entity_group': 'ORG',
  'score': 0.56734246,
  'word': '##wood',
  'start': 393,
  'end': 397},
 {'entity_group': 'MISC',
  'score': 0.38454476,
  'word': 'A',
  'start': 436,
  'end': 437},
 {'entity_group': '

In [ ]:
loc = [list(set([i['word'] for i in nlp(df.title[j] + df.text[j]) if i['entity_group'] == 'LOC' ])) for j in range(30)]

In [ ]:
loc

[['Hope Residence for Mothers with Children',
  "' s Village",
  'Hope Residence for Mothers with',
  "Eva ' s Village",
  'Paterson',
  'Washington Township'],
 ['Illinois', 'House', 'White House'],
 [],
 ['Britain', 'Asia', 'Kuala Lumpur'],
 ['Bratislava',
  'Košice',
  'Žilina',
  'Kamenné Square',
  'Slovakia',
  'Prešov',
  'Hotel Kyjev',
  'Nitra'],
 ['Beijing',
  'Supreme',
  'Tianjin',
  "' s Court",
  'China',
  'No. 2 Detention Centre'],
 ['Japan', 'Kansas City', 'U. S.'],
 [],
 ['Morocco', 'Rabat', 'U. S.', 'Sahara', 'United States'],
 ['Agriculture Park Wana',
  'Pine Nut Processing Plant',
  'Makeen',
  'Federally Administered Tribal Areas',
  'a Market at',
  'Makeen Market Complex ’',
  'Agriculture Park at',
  'South Waziristan Agency',
  'Pakistan',
  'Wana',
  'SW',
  'FA'],
 ['Cavan Institute',
  'Cavan Town Hall',
  'Ramor Theatre',
  'Ballyjamesduff',
  'Ballinagh Community Centre',
  'Courthouse Ballyjamesduff',
  'Crossdoney Presbyterian Hall',
  'Percy French Ho

In [ ]:
!pip install keybert -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.1 MB/s eta 0:00:00


In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT()
kw_model_multi = KeyBERT('paraphrase-multilingual-MiniLM-L12-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
df.text[0]+df.title[0]

In [ ]:
for i in range(len(loc)):
  tmp = [word.lower() for word in loc[i]]
  keywords = kw_model.extract_keywords((df.text[i]+df.title[i]).lower(), keyphrase_ngram_range=(1,  4), candidates=tmp, top_n=len(loc[i]))
  print(keywords)
  print('-'*10)

[('paterson', 0.3381), ('washington township', 0.1968)]
----------
[('white house', 0.215), ('house', 0.0406), ('illinois', -0.0219)]
----------
[]
----------
[('kuala lumpur', 0.3082), ('asia', 0.1424), ('britain', 0.0467)]
----------
[('bratislava', 0.3434), ('hotel kyjev', 0.3364), ('slovakia', 0.3233), ('kamenné square', 0.2457), ('žilina', 0.1825), ('košice', 0.1622), ('prešov', 0.1509), ('nitra', 0.1059)]
----------
[('tianjin', 0.435), ('beijing', 0.3983), ('china', 0.2546), ('supreme', 0.1254)]
----------
[('kansas city', 0.0995), ('japan', 0.0691)]
----------
[]
----------
[('morocco', 0.4525), ('sahara', 0.3695), ('rabat', 0.2827), ('united states', 0.2168)]
----------
[('south waziristan agency', 0.5384), ('federally administered tribal areas', 0.3946), ('pakistan', 0.3587), ('agriculture park wana', 0.3114), ('wana', 0.2054), ('pine nut processing plant', 0.0957), ('makeen', 0.0373)]
----------
[('ballinagh community centre', 0.3173), ('cavan institute', 0.2869), ('ireland'

In [ ]:
def final_task(text):
  candidate = (set([i['word'] for i in nlp(text) if i['entity_group'] == 'LOC' ]))
  tmp = [word.lower() for word in candidate]
  keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1,  4), candidates=tmp, top_n=4)
  cities = location_in_text(text)[1:]
  return [keywords, cities]

In [ ]:
# final_task(df.text[5])[0][0][0]
final_task("I am in a writer from Mumbai and today I am writing about the pollution in Delhi")

[[('delhi', 0.4495), ('mumbai', 0.398)], [[], ['Mumbai', 'Delhi']]]

# Converting to lat long

In [ ]:
import requests
coord = []
def getlatLong(sent):
  locs = final_task(sent)
  for i in locs[0]:
    url = f"https://api.geoapify.com/v1/geocode/search?text={i[0]}&format=json&apiKey=824e84df8379484ab446e01356b19e98"
    response = requests.get(url)
    a=response.json()['results']
    coord.append([a[0]['lat'], a[0]['lon']])
  return coord

In [ ]:
val = getlatLong("I am in a writer from Mumbai and today I am writing about the pollution in Delhi")

In [ ]:
!pip install folium -q
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

In [ ]:
import contextily as ctx

In [ ]:
import folium
from shapely.geometry import Point

def draw_map(coordinates):
    m = folium.Map(location=coordinates[0], zoom_start=13)

    for coord in coordinates:
        folium.Marker(coord).add_to(m)

    return m


In [ ]:
val

[[42.4297057, -91.3309112],
 [19.0785451, 72.878176],
 [42.4297057, -91.3309112],
 [19.0785451, 72.878176]]

In [ ]:
draw_map(val)